In [3]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
from utils import processLasairData

project_root = Path.cwd().parent
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

ztf_data = project_root / "ztf_cleansed.csv"

ztf_df = pd.read_csv(ztf_data)

print(ztf_df.head())

          ZTFID      IAUID           RA          Dec    peakt peakfilt  \
0  ZTF19aaagfme  SN2022okv  23:19:48.11  +07:06:02.2  1773.96        r   
1  ZTF19aaairqh   SN2019bp  06:22:21.88  +29:19:22.3   505.63        r   
2  ZTF19aaarhtg  SN2020pki  21:12:24.30  +13:00:17.1  1051.88        r   
3  ZTF19aabmybj   SN2019on  08:59:45.94  +47:49:34.0   503.01        r   
4  ZTF19aabvfwn   SN2019jf  08:36:14.79  -05:21:02.1   506.96        g   

   peakmag  peakabs  duration    rise    fade   type  redshift          b  \
0  17.9917   -18.96    28.221   8.577  19.644  SN Ia   0.05500  54.535723   
1  16.8079   -18.79    25.124  15.960   9.164  SN Ia   0.02200   7.188820   
2  18.0567   -17.56    18.006  11.999   6.007  SN Ia   0.02800 -23.438487   
3  17.1110   -19.14    24.490   9.229  15.261  SN Ia   0.04000  40.942318   
4  17.6411   -18.76    30.175  13.167  17.008  SN Ia   0.04138  20.409794   

     A_V  
0  0.072  
1  0.857  
2  0.248  
3  0.067  
4  0.110  


In [4]:
from typing import Any

lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    try:
        result = client.object(ztf_id)
    except Exception as e:
        print(f"\nLasair API error for {ztf_id}: {e}")
        continue

    if not result or 'candidates' not in result:
        print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
        continue
    if "error" in result:
        print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
        continue

    candidates = result['candidates']
    if not candidates:
        print(f"\nNo light curve data for {ztf_id}")
        continue

    output_rows = []
    for cand in candidates:
        # fid: filter ID (1=g, 2=r)
        fid = cand.get("fid")
        fid_to_letter = {1: "g", 2: "r", 3: "i"}
        filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None

        out_row = {
            'ztf_id': ztf_id,
            'iauid': iauid,
            'MJD': cand.get('mjd', None),
            'filter': filter_letter,
            'unforced_mag': cand.get('magpsf', None),
            'unforced_mag_error': cand.get('sigmapsf', None),
            'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
            'forced_ujy': cand.get('forcediffimflux', None),
            'forced_ujy_error': cand.get('forcediffimfluxunc', None),
        }
        output_rows.append(out_row)

    # Save per-object light curve CSV in run_folder
    df_obj = pd.DataFrame(output_rows)
    out_path = run_folder / f"{ztf_id}_lightcurve.csv"
    df_obj.to_csv(out_path, index=False)
    print(f" Saved {len(output_rows)} rows to {out_path.name}.", end=' ')
    try:
        processLasairData(df_obj, out_path)
        num_success += 1
        print(f"Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f"Error running processLasairData: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/9: ZTF19aaagfme... Saved 82 rows to ZTF19aaagfme_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF19aaagfme_lightcurve.csv (57 rows)
Done (1/1)

Processing   2/9: ZTF19aaairqh... Saved 41 rows to ZTF19aaairqh_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF19aaairqh_lightcurve.csv (30 rows)
Done (2/2)

Processing   3/9: ZTF19aaarhtg... Saved 115 rows to ZTF19aaarhtg_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF19aaarhtg_lightcurve.csv (25 rows)
Done (3/3)

Processing   4/9: ZTF19aabmybj... Saved 48 rows to ZTF19aabmybj_lightcurve.csv. Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/test2/ZTF19aabmybj_lightcurve.csv (24 rows)
Done (4/4)

Processing   5/9: ZTF19aabvfwn... Saved 20 rows to ZTF19aabvfwn_lightcurve.csv. Successfully cleaned and wrote ligh